In [ ]:
""" MonkeyPie
    
    FIND FUZZY DUPLICATES FOR SONGS ACROSS ANY CODEC
    - include check better quality, including probable glitch/skip analysis
    
    - matching inc. levenshtein fuzzy-compare of the "artist + songtitle"
        and of audio profiles
        
"""

""" PROCESS
    
    - run and create analysis-sets for all possible duplicates
        - is pkl with song ID, path [ and dbItem object? ]
    - sequentially place each analysis-set in now playing, 
        - first check now playing is empty (on start, not each dupset) and offer store as playlist
    - prompt user to play through all in the set and either:
        - remove undesired duplicates from now playing, and enter "keep remaining" into [console? popup? ]
            - marks remaining set as "non-duplicate" in a pkl db for future reference
            - move excluded out of library and filedir, to separate subdir in "dups_removed"
        - go next or quit
    
"""

In [ ]:
import win32com.client
# import sys
# import time
# import Levenshtein
# import codecs

In [ ]:
def get_songsDB(
    dispatch = 'SongsDB.SDBApplication',
    shutdown = False
    ):
        SDB = win32com.client.Dispatch( dispatch )  # check how interacts w. mmnk
        SDB.ShutdownAfterDisconnect = shutdown
        return SDB

def getResults_pathLike ( W32_SongsDB, matchStr ):
    results = []
    seltracks = W32_SongsDB.Database.QuerySongs( 
        f"Songs.SongPath LIKE '%{ matchStr }'")
        
    while not seltracks.EOF:
        resultItem = seltracks.Item
        results.append(resultItem)
        seltracks.Next()
        
    print( f"QUERY [ {matchStr} ] RETURNED [ { len(results)} ] RESULTS " )
    W32_SongsDB = None
    return results

In [ ]:
def main():
    
    SongsDB = get_songsDB()
    dbItems_flac = getResults_pathLike ( SongsDB, ".flac" )
    
    return dbItems_flac

# if __name__ == '__main__':
flacs = main()

# takes ~8 sec to return ~8000 results from ~44000 files

In [ ]:
# ANY song, path is like "G:\E MUSIC"

SongsDB = get_songsDB()
dbItems_flac = getResults_pathLike ( SongsDB, ".flac" )
len(dbItems_flac)

In [ ]:
SongsDB = get_songsDB()
dbItems_mp3 = getResults_pathLike ( SongsDB, ".mp3" )
len(dbItems_mp3)

In [ ]:
dbItems_wav = getResults_pathLike ( SongsDB, ".wav" )
len(dbItems_wav)

In [ ]:
len(dbItems_flac) + len(dbItems_mp3) + len(dbItems_wav)

In [ ]:
monkey_db_items = dbItems_flac + dbItems_mp3 + dbItems_wav

In [ ]:
# # Get track object attributes
# dirStrs = sorted( [ attrStr for attrStr in flacs[0].__dir__() ] )
# for dirStr in dirStrs: print( dirStr )

In [ ]:
attribsSelect = [
    'ID',
    'Title',
    'ArtistName',
    'SongLengthString',
    'AlbumName',
    'AlbumArtistName',
    'MusicComposer',
    'Encoder',
    'Bitrate',
    'SampleRate',
    'Channels',
    'FileLength',
    'Date',
    'LastPlayed',
    'DateAdded',
    'DateDBModified',
    'FileModified',
    'DiscNumber',
    'TrackOrder',
    'Genre',
    'Path',
    'PlayCounter',
    'SkipCount',
    'SongID',
    'SongLength',
]

In [ ]:
def getAttrValPair( obj, attString, toConsole=False ):
    
    value = eval( f"obj.{attString}" )
    
    if toConsole:
        valStr = str(value)[:60]
        if len(valStr) > 59: valStr += "..."
        padd = max([ len(attrStr) for attrStr in attribsSelect ]) + 2
        print( f"{attString}:{ ' ' * (padd-len(attString)) }{valStr}" )
        
    else: return ( attString, value )

In [ ]:
def fileLengthConvert( byteSize, mb = True ):
    # disagrees (less than by ~1mb) win filesys "size" field
    outSize = byteSize / 1024
    if mb: return outSize / 1024
    return outSize

In [ ]:

size = fileLengthConvert( flacs[0].FileLength )
size

In [ ]:
# output selected track attributes
for attrStr in attribsSelect: getAttrValPair( flacs[0], attrStr, toConsole=True )

In [ ]:
mm_lib = {}

for tr_obj in monkey_db_items:
    dct = { att : eval( f"tr_obj.{att}" ) for att in attribsSelect }
    mm_lib.update( { dct["ID"] : dct } )

len( mm_lib.keys() )

In [ ]:
import pandas as pd

In [ ]:

df = pd.DataFrame( mm_lib )




In [ ]:
df

In [ ]:
from OsOps import Ops

import pywintypes

pywintypes.datetime = pywintypes.TimeType

import os

In [ ]:
Ops().storePKL( df, "mm_lib_df", os.getcwd() )

In [ ]:
unpkl = Ops().unPklData( os.getcwd(), "mm_lib_df" )

In [ ]:
unpkl.keys()

In [ ]:
unpkl_lib = unpkl['mm_lib_df_230317_120649914451.pkl']

In [ ]:
unpkl_lib.shape

In [ ]:
# add to now playing
SDB = win32com.client.Dispatch('SongsDB.SDBApplication')
SDB.ShutdownAfterDisconnect = False
SDB.Player.PlaylistAddTrack( flacs[0] )
SDB = None